# Import packages

In [ ]:
import os
import cv2
import cmaps
import cmocean
import numpy as np
import xarray as xr
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
from netCDF4 import Dataset
from datetime import date
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import Polygon
from matplotlib import rcParams
from matplotlib.backends.backend_pdf import PdfPages
version = mpl.__version__
rcParams['font.family'] = 'sans-serif'
directory   = '/srv/scratch/z3533156'

# Import time functions

In [ ]:
def datestring_to_serial_day(datestring,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import pandas as pd
    import datetime
    serial_day_timedelta = pd.to_datetime(datestring) - datetime.datetime(epochY,epochm,epochd,epochH,epochM)
    corrected_serial_day_number = serial_day_timedelta.days + serial_day_timedelta.seconds/86400
    return corrected_serial_day_number
def serial_day_to_datestring(day,epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0):
    import datetime
    corrected_date = datetime.datetime(epochY,epochm,epochd,epochH,epochM) + datetime.timedelta(day)
    return corrected_date.strftime("%Y-%m")  

# Read mean SSTA

In [ ]:
AVISO_SSH            = Dataset(directory+'/MHW/AVISO/AVISO_EAC_monthly_SSH.nc','r')
AVISO_grid           = Dataset(directory+'/MHW/AVISO/AVISO_EAC_MHW.nc','r')
SSH                  = np.transpose(AVISO_SSH.variables['SSH'],[2,1,0])
dataset              = sio.loadmat(directory+'/MHW/Figure_data/Figure1_OSTIA_MHW.mat')
dataset1             = sio.loadmat(directory+'/MHW/Figure_data/Figure1_V10_MHW.mat')
V10_daily_ts         = dataset1['V10_daily_ts'][0,:]
lon0                 = dataset['lon_SST'][:,0]
lat0                 = dataset['lat_SST'][:,0]
lon1                 = AVISO_grid.variables['lon']
lat1                 = AVISO_grid.variables['lat']
time_daily           = dataset['time_daily'][:,0]
time_monthly         = dataset['time_monthly'][:,0]
SSTA_monthly         = dataset['SSTA_monthly']
SSTA_Dec             = dataset['SSTA_Dec'][0,:]
SSTA_Jan             = dataset['SSTA_Jan'][0,:]
SSTA_Feb             = dataset['SSTA_Feb'][0,:]
SST_daily_ts         = dataset['SST_daily_ts'][0,:]
SST_daily_clim_ts    = dataset['SST_daily_clim_ts'][0,:]
SST_daily_prctile_ts = dataset['SST_daily_prctile_ts'][0,:]
SSH_clim             = dataset['SSH_monthly_clim']
lon_SST              = np.tile(lon0,[np.size(lat0,0),1]).transpose()
lat_SST              = np.tile(lat0,[np.size(lon0,0),1])
lon_SSH              = np.tile(lon1,[np.size(lat1,0),1]).transpose()
lat_SSH              = np.tile(lat1,[np.size(lon1,0),1])
time                 = datestring_to_serial_day("1993-01-01",epochY=1990,epochm=1,epochd=1,epochH=0,epochM=0)
daily_t              = np.arange(date(1993,1,1).toordinal(),date(2022,12,31).toordinal()+1,1)
daily_dates          = [date.fromordinal(tt.astype(int)) for tt in daily_t]
monthly_t            = np.arange(date(1993,1,1).toordinal(),date(2022,12,31).toordinal()+1,365.5)
monthly_dates        = [date.fromordinal(tt.astype(int)) for tt in monthly_t]
dataset2             = sio.loadmat(directory+'/MHW/Figure_data/Figure_etopo_OSTIA.mat')
depth                = dataset2['depth'][:,:]
depth[lon_SST>155]   = np.nan

# Plot the spatial distribution and time series of SSTA 

In [ ]:
levels1       = np.linspace(-2,2,50)
tick_marks1   = np.linspace(-2,2,5)
levels2       = np.linspace(-0.5,0.5,50)
tick_marks2   = np.linspace(-0.5,0.5,6)
ssh_levels1   = np.linspace(0.949,0.951,2)
ssh_levels2   = np.linspace(0,0.5,6)
ssh_levels3   = np.linspace(-0.5,0,6)
cmaps1        = cmaps.cmocean_balance
fig_ratio     = 0.9
labelfont     = 40
padspacescale = 30
labelpadscale = 4
linefont      = 5
scale1        = 1.16
scale2        = 1.62
scale3        = 1.62
m_scale       = 1
fig           = plt.figure(figsize=(30, 20))
gs            = gridspec.GridSpec(2,3)
labels        = ['a','b','c','d']
#%%%%%%%%%%%%%%% Spatial distributions of SSTA %%%%%%%%%%%%%%%%%%%%%
for i in range(3):
    k=i+347
    if i==0:
        j=11
    else:
        j=i-1
    ax1 = fig.add_subplot(gs[i])
    l, b, w, h = ax1.get_position().bounds
    m          = Basemap(projection='merc',llcrnrlat=-40.0,urcrnrlat=-24+0.01,llcrnrlon=145.0,urcrnrlon=165,resolution='i')
    m.drawcoastlines(color='0.1',  linewidth=0.5*linefont)
    m.drawmapboundary(color='0.1', linewidth=0.5*linefont)
    m.fillcontinents(color='0.95', lake_color='white')
    m.drawmeridians(np.arange(140,165, 5),labels=[0,0,0,1],linewidth=0.5*linefont,dashes=[2,2],color='.7',fontsize=labelfont*0.8)
    plt.xlabel('Longitude',fontsize=1.0*labelfont,labelpad=-5,family='sans-serif')
    if i==0:
        plt.ylabel('Latitude',fontsize=1.0*labelfont,labelpad=1.5*padspacescale,family='sans-serif')
        m.drawparallels(np.arange(-40,-10, 2),labels=[1,0,0,0],linewidth=0.5*linefont,dashes=[2,2],color='.7',fontsize=labelfont*0.8)
    else:
        m.drawparallels(np.arange(-40,-10, 2),labels=[0,0,0,0],linewidth=0.5*linefont,dashes=[2,2],color='.7',fontsize=labelfont*0.8)
    x2, y2  = m(lon_SST, lat_SST)
    x3, y3  = m(lon_SSH, lat_SSH)
    lon2    = np.array([lon_SST[121,110],lon_SST[136,110],lon_SST[171,170],lon_SST[156,170],lon_SST[121,110]])
    lat2    = np.array([lat_SST[121,110],lat_SST[136,110],lat_SST[171,170],lat_SST[156,170],lat_SST[121,110]])
    x22,y22 = m(lon2,lat2)
    m.plot(x22,  y22,  linewidth=labelpadscale, linestyle='solid', color='green')
    cx0,  cy0  = m(146.05, -25.2)
    cx1,  cy1  = m(151.2093, -33.8688)
    cx11, cy11 = m(147.2093, -34.1688)
    plt.text(cx0, cy0,serial_day_to_datestring(time+time_monthly[k]), color='dodgerblue', fontsize=1.0*labelfont)
    if i==1:       
        plt.text(cx11, cy11,'Sydney', color='xkcd:red', fontsize=0.8*labelfont)
        m.plot(cx1, cy1, marker='^',  color='xkcd:red',   markersize=10)
        m.drawstates(linewidth=1.0*linefont, linestyle='dashed', color='xkcd:black', antialiased=1, ax=None, zorder=None)
    CB1 = m.contourf(x2, y2, SSTA_monthly[:,:,k],cmap=cmaps1,levels=levels1,origin='lower',extend='both')     
    CS1 = m.contour(x2, y2,  depth, np.arange(-201,-199,10), linewidths=1.0*linefont,linestyles='dashed',colors='xkcd:purple')
    CS2 = m.contour(x3, y3,  SSH[:,:,k],      ssh_levels1,linewidths=1.6*linefont,linestyles='solid',colors='black')
    CS3 = m.contour(x3, y3,  SSH_clim[:,:,j], ssh_levels1,linewidths=1.6*linefont,linestyles='solid',colors='gray')
    ax1.spines['bottom'].set_linewidth(labelpadscale)
    ax1.spines['left'].set_linewidth(labelpadscale)
    ax1.spines['top'].set_linewidth(labelpadscale)
    ax1.spines['right'].set_linewidth(labelpadscale)
    ax1.set_position([l, b, scale1*w, scale1*h])
    plt.title(labels[i], fontsize=1.0*labelfont,loc='left',pad=0.8*padspacescale, weight='bold',family='sans-serif')
    plt.tick_params(axis='both',which='major',bottom='on',left='on',length=50.0,width=20,colors='black',direction='out')
    cbaxes1     = fig.add_axes([0.13, 0.618, 0.01, 0.25])
    cb1 = plt.colorbar(CB1,orientation='vertical',cax = cbaxes1)
    cb1.set_ticks(tick_marks1)
    cb1.set_label(r'SSTA ($^\circ$C)', fontsize=0.8*labelfont,labelpad=-4)
    cb1.ax.tick_params(labelsize=0.8*labelfont)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
dt  = SST_daily_prctile_ts-SST_daily_clim_ts
dt1 = SST_daily_prctile_ts+dt
dt2 = SST_daily_prctile_ts+2*dt
dt3 = SST_daily_prctile_ts+3*dt
dt4 = SST_daily_prctile_ts+4*dt
ax2 = fig.add_subplot(gs[3:5])
l, b, w, h = ax2.get_position().bounds
legends = ['SST','Clim','Threshold']
plt.plot(daily_dates, SST_daily_ts,         color='xkcd:forest green',    linewidth=1.6*linefont,linestyle='solid',zorder=3)
plt.plot(daily_dates, SST_daily_clim_ts,    color='darkgray', linewidth=2*linefont)
plt.plot(daily_dates, SST_daily_prctile_ts, color='black',    linewidth=1.6*linefont)
plt.legend(legends, loc = 2, ncol = 3, fontsize=30, facecolor='xkcd:pale blue', frameon=True, framealpha=0.8)
plt.plot(daily_dates, dt1, color='black',    linewidth=1*linefont)
plt.plot(daily_dates, dt2, color='black',    linewidth=1*linefont)
plt.plot(daily_dates, dt3, color='black',    linewidth=1*linefont)
plt.fill_between(daily_dates[10573:10600], 18, 28, facecolor='xkcd:cool grey', alpha=0.4)
plt.fill_between(daily_dates[10620:10648], 18, 28, facecolor='xkcd:cool grey', alpha=0.4)
plt.fill_between(daily_dates,dt2,dt3, where=dt3>dt2,color='xkcd:red orange',zorder=1)
plt.fill_between(daily_dates,dt1,dt2, where=dt2>dt1,color='xkcd:orange',zorder=1)
plt.fill_between(daily_dates,SST_daily_prctile_ts,dt1, where=dt1>SST_daily_prctile_ts,color='xkcd:pale yellow',zorder=1)
plt.title(labels[3], fontsize=1.0*labelfont,loc='left', pad=0.8*padspacescale, weight='bold',family='sans-serif')
plt.text(daily_dates[10502],22.25, 'III', fontsize=1.0*labelfont, color='xkcd:black')
plt.text(daily_dates[10503],21.25, 'II', fontsize=1.0*labelfont, color='xkcd:black')
plt.text(daily_dates[10504],20.35, 'I', fontsize=1.0*labelfont, color='xkcd:black')
plt.text(daily_dates[10506],22.3, 'Severe', fontsize=0.85*labelfont, color='xkcd:black')
plt.text(daily_dates[10506],21.3, 'Strong', fontsize=0.85*labelfont, color='xkcd:black')
plt.text(daily_dates[10506],20.45, 'Moderate', fontsize=0.85*labelfont, color='xkcd:black')
plt.xlim(daily_dates[10500], daily_dates[10682])
plt.ylim(18,28)
xminorLocator   = mpl.dates.WeekdayLocator(byweekday=4,interval=1)
xmajorLocator   = mpl.dates.WeekdayLocator(byweekday=4,interval=1)
xmajorFormatter = mpl.dates.DateFormatter('%Y-%m-%d')
yminorLocator   = plt.MultipleLocator(1)
ax2.xaxis.set_minor_locator(xminorLocator)
ax2.xaxis.set_major_locator(xmajorLocator)
ax2.xaxis.set_major_formatter(xmajorFormatter)
ax2.axes.xaxis.set_tick_params(pad=3*labelpadscale)
ax2.yaxis.set_minor_locator(yminorLocator)
ax2.axes.yaxis.set_tick_params(pad=3*labelpadscale)
ax2.spines['bottom'].set_linewidth(labelpadscale)
ax2.spines['left'].set_linewidth(labelpadscale)
ax2.spines['top'].set_linewidth(labelpadscale)
ax2.spines['right'].set_linewidth(labelpadscale)
plt.xticks(fontsize=labelfont*0.8,rotation=75)
plt.xlabel('Date',fontsize=labelfont*0.9,labelpad=5)
plt.ylabel(r'Daily SST ($^\circ$C)',fontsize=labelfont*0.9,labelpad=5)
plt.yticks(np.linspace(18,28,6), ['18', '20', '22', '24', '26', '28'],fontsize=0.8*labelfont)
ax2.tick_params(axis='x',which='minor',bottom='on',left='on',top='on',right='on',length=8.0,width=5,colors='black',direction='in')
ax2.tick_params(axis='x',which='major',bottom='on',left='on',top='on',right='on',length=15.0,width=5,colors='black',direction='in')
ax2.tick_params(axis='y',which='minor',bottom='on',left='on',top='on',right='on',length=8.0,width=5,colors='black',direction='in')
ax2.tick_params(axis='y',which='major',bottom='on',left='on',top='on',right='on',length=15.0,width=5,colors='black',direction='in')
####################################################################################
ax2.set_position([l, b, scale2*w, 0.6*scale2*h])
fig.savefig(directory+'/MHW/Figure_plots/Figure1_MHWs_SSTA.png',dpi=300,bbox_inches = 'tight')